In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys
import torch
from torch.utils.data import DataLoader, Dataset
import pymc as pm
from properscoring import crps_ensemble
import random
from patsy import dmatrix
import arviz as az
import matplotlib.pyplot as plt



sys.path.append(os.path.abspath('../src'))
project_dir = Path.cwd().parent

from data_tools.data_utils import SeroDataset, PartialCountDataset, TrueCountDataset
from model_tools.train_utils import BaseTrain, SparsePropTrain, SeroTrain
from model_tools.models import NowcastPNN, PropPNN, SparsePropPNN, SeroPNN
from model_tools.evaluation import eval_pnn, eval_prop_pnn, eval_sparse_prop_pnn, eval_sero_pnn, plot_pnn_preds, plot_prop_pnn_preds, plot_sero_pnn_preds

M = 50
D = 40

start_year = 2022
end_year = 2022
seed = 123

# 123, 2019, 2023, 15

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False




In [2]:
# Create count obj
delays_df = pd.read_csv(project_dir / "data" / "transformed" / "DENG_delays.csv")
delays_df['Collection date'] = pd.to_datetime(delays_df['Collection date'])

partial_count_dataset = PartialCountDataset(delays_df, D=D, M=M, norm=False)
true_count_dataset = TrueCountDataset(delays_df)


In [3]:
# End of 2023 appears to have some incomplete data
if end_year == 2023:
    dates = list(pd.date_range(f"{start_year}-01-01",f"{end_year}-12-25", freq='D'))
else:
    dates = list(pd.date_range(f"{start_year}-01-01",f"{end_year}-12-31", freq='D'))

In [4]:
dates

[Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-01-02 00:00:00'),
 Timestamp('2022-01-03 00:00:00'),
 Timestamp('2022-01-04 00:00:00'),
 Timestamp('2022-01-05 00:00:00'),
 Timestamp('2022-01-06 00:00:00'),
 Timestamp('2022-01-07 00:00:00'),
 Timestamp('2022-01-08 00:00:00'),
 Timestamp('2022-01-09 00:00:00'),
 Timestamp('2022-01-10 00:00:00'),
 Timestamp('2022-01-11 00:00:00'),
 Timestamp('2022-01-12 00:00:00'),
 Timestamp('2022-01-13 00:00:00'),
 Timestamp('2022-01-14 00:00:00'),
 Timestamp('2022-01-15 00:00:00'),
 Timestamp('2022-01-16 00:00:00'),
 Timestamp('2022-01-17 00:00:00'),
 Timestamp('2022-01-18 00:00:00'),
 Timestamp('2022-01-19 00:00:00'),
 Timestamp('2022-01-20 00:00:00'),
 Timestamp('2022-01-21 00:00:00'),
 Timestamp('2022-01-22 00:00:00'),
 Timestamp('2022-01-23 00:00:00'),
 Timestamp('2022-01-24 00:00:00'),
 Timestamp('2022-01-25 00:00:00'),
 Timestamp('2022-01-26 00:00:00'),
 Timestamp('2022-01-27 00:00:00'),
 Timestamp('2022-01-28 00:00:00'),
 Timestamp('2022-01-

In [5]:
class PropBNNDataset(Dataset):
    def __init__(self, partial_count_obj, true_count_obj, dates):
        self.partial_count_obj = partial_count_obj
        self.true_count_obj = true_count_obj
        self.dates = dates
    
    def __len__(self):
        return len(self.dates)
    
    def __getitem__(self, index):
        date = self.dates[index]
        window_dates = [date - pd.Timedelta(days=i) for i in range(M)]
        window_dates = sorted(window_dates)
        Z_obs = self.partial_count_obj.get_obs(date)
        y_true = [self.true_count_obj.get_y(day) for day in window_dates]
        y_true = np.array(y_true).reshape(M)
        dow = date.day_of_week
        return Z_obs, y_true, window_dates

set_seed(seed)
dataset = PropBNNDataset(partial_count_dataset, true_count_dataset, dates)




In [6]:
def silu(x):
    return x * pm.math.sigmoid(x)


def sampler_kwargs():
    return dict(
        nuts_sampler="nutpie",
        cores=1,
        init="adapt_diag",
        chains=1,
        draws=500,
        tune=500,
        target_accept=0.95,
        max_treedepth=10,
        nuts_sampler_kwargs={"backend": "jax", "gradient_backend": "jax"}
    )

def get_mask(D):
        mask_matrix = np.ones(shape=(D, D), dtype=bool)
        for i in range(D):
            for j in range(D):
                if i + j > D - 1:
                    mask_matrix[i, j] = False
        return mask_matrix

def create_fourier_features(t, n, p=10.0):
    x = 2 * np.pi * (np.arange(n) + 1) * t[:, None] / p
    return np.concatenate((np.cos(x), np.sin(x)), axis=1)


In [7]:
Z_obs, y_true, window_dates = dataset.__getitem__(22)

# Constants
t = np.arange(0, M)
t_week = t % 7
t_norm = t / M

n = 14
fourier_basis_biweek = create_fourier_features(t, n=n, p=3.5)
fourier_basis_week = create_fourier_features(t, n=n, p=7)


fourier_basis_biweek = fourier_basis_biweek - fourier_basis_biweek.mean(0, keepdims=True)
fourier_basis_week = fourier_basis_week - fourier_basis_week.mean(0, keepdims=True)

spline_trend = dmatrix(
    "bs(t, df=14, degree=3, include_intercept=True)", {"t": t_norm}, return_type='dataframe'
)
X_trend = np.asarray(spline_trend)

spline_week = dmatrix(
    "cc(t_week, df=7)", {"t_week": t_week}, return_type='dataframe'
)
X_week = np.asarray(spline_week)

Z_norm = Z_obs / (Z_obs.max() + 1e-6)

t_input = np.arange(M)[:, None] / M
time_input = np.concatenate([t_input, fourier_basis_biweek, fourier_basis_week], axis=1)



In [8]:
mask = np.ones_like(Z_obs, dtype=bool)
mask[-D:,:] = get_mask(D)
mask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [9]:
with pm.Model() as model:
    log_const = pm.Normal("const", sigma=10)
    

    # Trend spline coefficients
    sigma_trend = pm.HalfNormal("sigma_trend", 10)
    beta_trend = pm.Normal("beta_trend", mu=0, sigma=sigma_trend, shape=X_trend.shape[1])
    
    # Cyclic spline coefficients (seasonality)
    sigma_week = pm.HalfNormal("sigma_week", 10)
    beta_week = pm.Normal("beta_week", mu=0, sigma=sigma_week, shape=X_week.shape[1])
    
    # GRW for prediction intervals
    init_dist = pm.Normal.dist(mu=0, sigma=1)
    sigma_rw = pm.HalfNormal("sigma_rw", 1)
    rw = pm.GaussianRandomWalk("log_lambda", sigma=sigma_rw, shape=M, init_dist=init_dist)

    # Create lam param 
    log_lam = pm.Deterministic(
            "log_lam", rw + pm.math.dot(X_trend, beta_trend) + pm.math.dot(X_week, beta_week)
        )
    lam = pm.Deterministic("lam", pm.math.exp(log_const + log_lam))

    # NN for proportions from time inputs
    n_hidden = 16
    net_sd = 0.15
    
    W1 = pm.Normal("W1", 0, net_sd, shape=(1+4*n, n_hidden))
    b1 = pm.Normal("b1", 0, net_sd, shape=(n_hidden,))
    h1 = silu(pm.math.dot(time_input, W1) + b1)

    Wz = pm.Normal("Wz", 0, net_sd, shape=(D, n_hidden))
    bz = pm.Normal("bz", 0, net_sd, shape=(n_hidden,))
    hz = silu(pm.math.dot(Z_norm, Wz) + bz)
    h1 = hz + h1
    
    # ----- Output layer: hidden → delay bins -----
    W2 = pm.Normal("W2", 0, net_sd, shape=(n_hidden, D))
    b2 = pm.Normal("b2", 0, net_sd, shape=(D,))
    p_raw = pm.math.dot(h1, W2) + b2 
    p = pm.Deterministic("p", pm.math.softmax(p_raw, axis=1))


    # Make mu param per tim/delay point
    log_mu = pm.math.log(lam[:, None]) + pm.math.log(p + 1e-6)
    mu = pm.Deterministic("mu", pm.math.exp(log_mu))

    # Overdispersion per time point
    sigma_theta = pm.HalfNormal("sigma_theta", 2)
    theta = pm.Exponential("theta", lam=1/sigma_theta, shape=(M,1))
    theta_broadcast = np.repeat(theta, D, axis=1)

    # Observed data
    z = pm.NegativeBinomial("z", mu[mask], theta_broadcast[mask], observed=Z_obs[mask])
    idata = pm.sample(progressbar=True, **sampler_kwargs())


In [10]:
mu_samples = az.extract(idata, group='posterior', num_samples=1000)['mu']
theta_samples = az.extract(idata, group='posterior', num_samples=1000)['theta']

r = np.broadcast_to(theta_samples, mu_samples.shape)
mu = mu_samples
p = r / (r + mu)

z_samples = np.random.negative_binomial(r, p, size=mu_samples.shape)
z_samples.shape
y_preds = z_samples.sum(1)
y_preds.shape

(50, 500)

In [11]:
y_preds_med = np.quantile(y_preds, 0.5, axis=1)
y_preds_lower = np.quantile(y_preds, 0.025, axis=1)
y_preds_upper = np.quantile(y_preds, 0.975, axis=1)

plt.plot(window_dates, y_true, label='y')

plt.plot(window_dates, y_preds_med, label='BNN', color='green')
plt.fill_between(window_dates, y_preds_lower, y_preds_upper, color='green', alpha=0.2, label='BNN 95% CI')


plt.legend()
plt.tight_layout()
plt.title("Number of Dengue Fever Cases in Sao Paulo, Brazil")
plt.xlabel("Date of First Symptom")
plt.ylabel("Case Count")
plt.xticks(rotation=45)
plt.show()

In [12]:
mu_post_samples = az.extract(idata, group='posterior', num_samples=1000)['p']
mu_post_samples.mean("sample")[:, 0]

<xarray.DataArray 'p' (p_dim_0: 50)> Size: 400B
array([0.07167429, 0.11803198, 0.11296145, 0.12014769, 0.12746874,
       0.10948436, 0.07112028, 0.0724912 , 0.12044326, 0.11463107,
       0.12265543, 0.12947405, 0.11028177, 0.0719903 , 0.07276423,
       0.12090876, 0.11360296, 0.11930626, 0.12642636, 0.10864204,
       0.07173096, 0.07259902, 0.12000472, 0.11368859, 0.11738708,
       0.12176641, 0.09806632, 0.06430286, 0.06978237, 0.11437321,
       0.1089869 , 0.11517356, 0.12217397, 0.10285811, 0.06783892,
       0.06811621, 0.1139604 , 0.10848051, 0.1139604 , 0.12259875,
       0.10327867, 0.06831371, 0.06770194, 0.11203052, 0.10938396,
       0.11511991, 0.12261284, 0.09828559, 0.06411712, 0.06044055])
Coordinates:
  * p_dim_0  (p_dim_0) int64 400B 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
    p_dim_1  int64 8B 0